In [1]:
import nibabel as nib
import os
import numpy as np
import pandas as pd
from skimage import color
import matplotlib.pyplot as plt
from IPython.display import clear_output
import tensorflow as tf
import random
import pickle

In [ ]:
# Turning on GPU to load data
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

device = tf.device(device_name)

SystemError: ignored

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Functions

In [3]:
#Get the list of all the data

def get_17_paths(pathname):
  paths_list = []
  for rootname, dirnames, filenames in os.walk(pathname):
    for dirname in dirnames:
      if "Brats" in dirname:
        paths_list.append(os.path.join(rootname, dirname))
  return paths_list

In [4]:
def get_nii_path(pathname):
  paths_list = []
  for filename in os.listdir(pathname):
    if ".nii" in filename:
      paths_list.append(os.path.join(pathname, filename))
  return paths_list

In [5]:
class Sample:
    def __init__(self, flair, T1, T1c, T2, label, roi):
        self.flair = flair
        self.T1 = T1
        self.T1c = T1c
        self.T2 = T2
        self.label = label
        self.roi = roi

    def to_numpy(self):
      return np.array([self.flair, self.T1, self.T1c, self.T2, self.label])

    def from_numpy(self, np_array):
      self.flair = np_array[0]
      self.T1 = np_array[1]
      self.T1c = np_array[2]
      self.T2 = np_array[3]
      self.label = np_array[4]

In [18]:
'''
Given the nii paths for one data point, process the data 
'''
def process_nii_data(nii_paths):
  data = Sample(None, None, None, None, None, None)
  for nii in nii_paths:
    img_arr = nib.load(nii).get_fdata()
    if 'seg' in nii:
      data.label = img_arr
    elif 'flair' in nii:
      data.flair = img_arr
    elif 't1ce' in nii:
      data.T1c = img_arr
    elif 't1' in nii:
      data.T1 = img_arr
    elif 't2' in nii:
      data.T2 = img_arr
  return data

In [7]:
'''
Gets roi segmentation label
'''
def get_roi(nii_paths):
  data = Sample(None, None, None, None, None, None)
  for nii in nii_paths:
    img_arr = nib.load(nii).get_fdata()
    if 'seg' in nii:
      data.label = img_arr
    elif 'ROI' in nii:
      data.roi = img_arr
  return data

In [8]:
"""
This function removes all blank images from our data
Input: one nii
Output: mha without the blanks
"""
def remove_blanks(nii):
    # lpad, rpad = 1, 1
    label_zero_mask = nii.label.sum(axis = (0,1)) > 0
    

    # lidx, ridx = np.argmax(label_zero_mask), label_zero_mask.shape[0] - np.argmax(label_zero_mask[::-1]) - 1
    
    # label_zero_mask[lidx - lpad:lidx] = True
    # label_zero_mask[ridx + 1:ridx + 1 + rpad] = True

    nii.label = nii.label[:, :, label_zero_mask]

    nii.flair = nii.flair[:, :, label_zero_mask]

    nii.T1c = nii.T1c[:, :, label_zero_mask]

    nii.T1 = nii.T1[:, :, label_zero_mask]

    nii.T2 = nii.T2[:, :, label_zero_mask]

    rand_idxs = np.random.choice(np.arange(nii.label.shape[2]), 3, replace = False)

    nii.label = nii.label[:, :, rand_idxs]

    nii.flair = nii.flair[:, :, rand_idxs]

    nii.T1c = nii.T1c[:, :, rand_idxs]

    nii.T1 = nii.T1[:, :, rand_idxs]

    nii.T2 = nii.T2[:, :, rand_idxs]

    return nii

In [9]:
def remove_blank_roi(nii):
  lpad, rpad = 1, 1
  label_zero_mask = nii.label.sum(axis = (0,1)) > 0
  lidx, ridx = np.argmax(label_zero_mask), label_zero_mask.shape[0] - np.argmax(label_zero_mask[::-1]) - 1
  
  label_zero_mask[lidx - lpad:lidx] = True
  label_zero_mask[ridx + 1:ridx + 1 + rpad] = True
  
  nii.roi = nii.roi[:, :, label_zero_mask]
  return nii

In [28]:
'''
1. split into input and label
2. considered flair, t1, t1c, t2 as multiple inputs
3. repeated the label 4 times to match the inputs
4. considered each of the frames as inputs
'''
def process_samples(data):
  data = np.swapaxes(data, 2, 3)
  data = np.swapaxes(data, 1, 2)
  X = data[:4,:,:,:]
  y = data[4,:,:,:]
  y = np.reshape(y, (1, -1, 240, 240))
  y = np.repeat(y, 4, axis=0)
  # print(y.shape)
  # print(y[:, 23, 150:170, 150:170])
  return X, y

In [11]:
def three_channels(X, y):
  X = np.stack((X,)*3, axis=-1)
  y = np.stack((y,)*3, axis=-1)
  return X, y

In [12]:
def normalize_data(X):
  X = X/np.max(X)
  return X

# Getting Data

In [13]:
base_path = "/content/drive/MyDrive/Bioimaging/Bioimagining Project/data"
HGG_path = os.path.join(base_path, "BRATS2017", "Brats17TrainingData", "HGG")
LGG_path = os.path.join(base_path, "BRATS2017", "Brats17TrainingData", "LGG")

In [14]:
HGG_paths = get_17_paths(HGG_path)
LGG_paths = get_17_paths(LGG_path)

In [ ]:
print("HGG:", len(HGG_paths))
print("LGG:", len(LGG_paths))

HGG: 210
LGG: 75


In [15]:
#Testing get_nii_path
ex_train_nii_HGG = get_nii_path(HGG_paths[16])
# ex_train_nii_HGG_1 = get_nii_path(HGG_paths[100])
# ex_train_nii_HGG_2 = get_nii_path(HGG_paths[1])
# ex_train_nii_HGG_3 = get_nii_path(HGG_paths[30])
# ex_train_nii_HGG_4 = get_nii_path(HGG_paths[9])

In [ ]:
# ex_data = get_roi(ex_train_nii_HGG)
# ex_data_1 = get_roi(ex_train_nii_HGG_1)
# ex_data_2 = get_roi(ex_train_nii_HGG_2)
# ex_data_3 = get_roi(ex_train_nii_HGG_3)
# ex_data_4 = get_roi(ex_train_nii_HGG_4)

In [ ]:
print(ex_data.roi[150:170, 150:170, 30])


[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [ ]:
for i in range(len(HGG_paths)):
  if "2013" in HGG_paths[i]:
    print(i)

In [19]:
ex_data = process_nii_data(ex_train_nii_HGG)
# ex_data_1 = process_nii_data(ex_train_nii_HGG_1)
# ex_data_2 = process_nii_data(ex_train_nii_HGG_2)
# ex_data_3 = process_nii_data(ex_train_nii_HGG_3)
# ex_data_4 = process_nii_data(ex_train_nii_HGG_4)

In [20]:
ex_data = remove_blanks(ex_data)
# ex_data_1 = remove_blanks(ex_data_1)

In [22]:
print(ex_data.label.shape)

(240, 240, 3)


In [ ]:

# Playing video of label
for i in range(ex_data.roi.shape[2]):
  clear_output()
  plt.imshow(ex_data.roi[:, :, i])
  plt.pause(0.1)

KeyboardInterrupt: ignored

In [ ]:
ex_data = ex_data.to_numpy()
ex_data_1 = ex_data_1.to_numpy()
full_ex_data = np.concatenate((ex_data_1, ex_data), axis = 3)

In [ ]:
print(full_ex_data.shape)

(5, 240, 240, 125)


In [ ]:
full_ex_data_X, full_ex_data_y = process_samples(full_ex_data)
# print(full_ex_data_X.shape)

(4, 125, 240, 240)
[[[2. 2. 2. ... 4. 4. 1.]
  [2. 2. 2. ... 4. 1. 1.]
  [2. 2. 2. ... 1. 1. 1.]
  ...
  [2. 2. 2. ... 4. 4. 4.]
  [2. 2. 2. ... 4. 4. 4.]
  [2. 2. 2. ... 4. 4. 4.]]

 [[2. 2. 2. ... 4. 4. 1.]
  [2. 2. 2. ... 4. 1. 1.]
  [2. 2. 2. ... 1. 1. 1.]
  ...
  [2. 2. 2. ... 4. 4. 4.]
  [2. 2. 2. ... 4. 4. 4.]
  [2. 2. 2. ... 4. 4. 4.]]

 [[2. 2. 2. ... 4. 4. 1.]
  [2. 2. 2. ... 4. 1. 1.]
  [2. 2. 2. ... 1. 1. 1.]
  ...
  [2. 2. 2. ... 4. 4. 4.]
  [2. 2. 2. ... 4. 4. 4.]
  [2. 2. 2. ... 4. 4. 4.]]

 [[2. 2. 2. ... 4. 4. 1.]
  [2. 2. 2. ... 4. 1. 1.]
  [2. 2. 2. ... 1. 1. 1.]
  ...
  [2. 2. 2. ... 4. 4. 4.]
  [2. 2. 2. ... 4. 4. 4.]
  [2. 2. 2. ... 4. 4. 4.]]]


In [ ]:
#Test Visualize 
# ex_data.label = np.stack((ex_data.label,)*3, axis=-1)
# plt.imshow(ex_data.flair[:, :, 4])
plt.imshow(ex_data.label[:, :, 99])
# print(ex_data.label.shape)
# print(ex_data.flair[70:110, 130:170, 105]/np.max(ex_data.flair))
print(ex_data.label[100:170, 100:170, 99])

# # Playing video of label
# for i in range(ex_data.label.shape[2]):
#   clear_output()
#   print(i)
#   plt.imshow(ex_data.label[:, :, i])
#   plt.pause(0.01)

# Getting Average Count of Blanks

In [ ]:
print(type(ex_data.flair))

<class 'numpy.memmap'>


In [ ]:
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_test", "test_dataset_HGG_0.npy")
example_test_open_file = np.load(test_dataset_path)

In [ ]:
label_zero_mask = ex_data.label.sum(axis = (0,1)) > 0
print(label_zero_mask)
lpad, rpad = 1, 1
lidx, ridx = np.argmax(label_zero_mask), label_zero_mask.shape[0] - np.argmax(label_zero_mask[::-1]) - 1
label_zero_mask[lidx - lpad:lidx] = True
label_zero_mask[ridx + 1:ridx + 1 + rpad] = True
print(label_zero_mask)

In [ ]:
label_zero_mask = ex_data_1.label.sum(axis = (0,1)) > 0
print(label_zero_mask.sum())

72


In [ ]:
label_zero_mask = ex_data_2.label.sum(axis = (0,1)) > 0
print(label_zero_mask.sum())

45


In [ ]:
label_zero_mask = ex_data_3.label.sum(axis = (0,1)) > 0
print(label_zero_mask.sum())

62


In [ ]:
label_zero_mask = ex_data_4.label.sum(axis = (0,1)) > 0
print(label_zero_mask.sum())

65


In [ ]:
print(example_test_open_file.shape)

# Splitting Data and Saving Paths

# Mix Split Paths

In [25]:
base_path = "/content/drive/MyDrive/Bioimaging/Bioimagining Project/data"
paths_path = os.path.join(base_path, "BRATS2017", "paths")

In [26]:
num_split = 50
pathlst_mix_train = [os.path.join(paths_path, "mix_train_lst_{}.pkl".format(i)) for i in range(num_split)]
pathlst_mix_val = [os.path.join(paths_path, "mix_val_lst_{}.pkl".format(i)) for i in range(num_split)]
pathlst_mix_test = [os.path.join(paths_path, "mix_test_lst_{}.pkl".format(i)) for i in range(num_split)]

# Put Mix into Files

In [ ]:
# Split train into 60/20/20 for train/validation/test
mix_paths = HGG_paths + LGG_paths
random.shuffle(mix_paths)

paths_mix_train = mix_paths[:int((len(mix_paths)+1)*.60)]
paths_mix_val = mix_paths[int((len(mix_paths)+1)*.60):int((len(mix_paths)+1)*.80)]
paths_mix_test = mix_paths[int((len(mix_paths)+1)*.80):]

In [ ]:
print("number of HGG train", len(paths_mix_train))
print("number of HGG val", len(paths_mix_val))
print("number of HGG test", len(paths_mix_test))

number of HGG train 171
number of HGG val 57
number of HGG test 57


In [ ]:
paths_mix_train_lst = [paths_mix_train[int((len(paths_mix_train))*(1/num_split) * (i)):int((len(paths_mix_train))*(1/num_split) * (i + 1))] for i in range(num_split)]

for i in range(num_split): 
  with open(pathlst_mix_train[i] , 'wb') as fp:
      pickle.dump(paths_mix_train_lst[i], fp)

In [ ]:
paths_mix_val_lst = [paths_mix_val[int((len(paths_mix_val))*(1/num_split) * (i)):int((len(paths_mix_val))*(1/num_split) * (i + 1))] for i in range(num_split)]

for i in range(num_split): 
  with open(pathlst_mix_val[i] , 'wb') as fp:
      pickle.dump(paths_mix_val_lst[i], fp)

In [ ]:
paths_mix_test_lst = [paths_mix_test[int((len(paths_mix_test))*(1/num_split) * (i)):int((len(paths_mix_test))*(1/num_split) * (i + 1))] for i in range(num_split)]

for i in range(num_split): 
  with open(pathlst_mix_test[i] , 'wb') as fp:
      pickle.dump(paths_mix_test_lst[i], fp)

# HGG Split Paths

In [ ]:
pathlst_HGG_train_0 = os.path.join(paths_path, "HGG_train_lst_0.pkl")
pathlst_HGG_train_1 = os.path.join(paths_path, "HGG_train_lst_1.pkl")
pathlst_HGG_train_2 = os.path.join(paths_path, "HGG_train_lst_2.pkl")
pathlst_HGG_train_3 = os.path.join(paths_path, "HGG_train_lst_3.pkl")
pathlst_HGG_train_4 = os.path.join(paths_path, "HGG_train_lst_4.pkl")
pathlst_HGG_train_5 = os.path.join(paths_path, "HGG_train_lst_5.pkl")
pathlst_HGG_train_6 = os.path.join(paths_path, "HGG_train_lst_6.pkl")
pathlst_HGG_train_7 = os.path.join(paths_path, "HGG_train_lst_7.pkl")
pathlst_HGG_train_8 = os.path.join(paths_path, "HGG_train_lst_8.pkl")
pathlst_HGG_train_9 = os.path.join(paths_path, "HGG_train_lst_9.pkl")

In [ ]:
pathlst_HGG_val_0 = os.path.join(paths_path, "HGG_val_lst_0.pkl")
pathlst_HGG_val_1 = os.path.join(paths_path, "HGG_val_lst_1.pkl")
pathlst_HGG_val_2 = os.path.join(paths_path,  "HGG_val_lst_2.pkl")
pathlst_HGG_val_3 = os.path.join(paths_path, "HGG_val_lst_3.pkl")
pathlst_HGG_val_4 = os.path.join(paths_path, "HGG_val_lst_4.pkl")
pathlst_HGG_val_5 = os.path.join(paths_path,  "HGG_val_lst_5.pkl")
pathlst_HGG_val_6 = os.path.join(paths_path, "HGG_val_lst_6.pkl")
pathlst_HGG_val_7 = os.path.join(paths_path, "HGG_val_lst_7.pkl")
pathlst_HGG_val_8 = os.path.join(paths_path,  "HGG_val_lst_8.pkl")
pathlst_HGG_val_9 = os.path.join(paths_path, "HGG_val_lst_9.pkl")

In [ ]:
pathlst_HGG_test_0 = os.path.join(paths_path, "HGG_test_lst_0.pkl")
pathlst_HGG_test_1 = os.path.join(paths_path, "HGG_test_lst_1.pkl")
pathlst_HGG_test_2 = os.path.join(paths_path, "HGG_test_lst_2.pkl")
pathlst_HGG_test_3 = os.path.join(paths_path, "HGG_test_lst_3.pkl")
pathlst_HGG_test_4 = os.path.join(paths_path, "HGG_test_lst_4.pkl")
pathlst_HGG_test_5 = os.path.join(paths_path, "HGG_test_lst_5.pkl")
pathlst_HGG_test_6 = os.path.join(paths_path, "HGG_test_lst_6.pkl")
pathlst_HGG_test_7 = os.path.join(paths_path, "HGG_test_lst_7.pkl")
pathlst_HGG_test_8 = os.path.join(paths_path, "HGG_test_lst_8.pkl")
pathlst_HGG_test_9 = os.path.join(paths_path, "HGG_test_lst_9.pkl")

# Put HGG into Files

In [ ]:
print("number of HGG train", len(paths_HGG_train))
print("number of HGG val", len(paths_HGG_val))
print("number of HGG test", len(paths_HGG_test))

number of HGG train 126
number of HGG val 42
number of HGG test 42


In [ ]:
# Split train into 60/20 for train/validation/test
random.shuffle(HGG_paths)

paths_HGG_train = HGG_paths[:int((len(HGG_paths)+1)*.60)]
paths_HGG_val = HGG_paths[int((len(HGG_paths)+1)*.60):int((len(HGG_paths)+1)*.80)]
paths_HGG_test = HGG_paths[int((len(HGG_paths)+1)*.80):]

In [ ]:
# Split HGG data into 1/4 to make it more manageable to run 
# paths_HGG_train_0 = paths_HGG_train[:int((len(paths_HGG_train)+1)*.1)]
# paths_HGG_train_1 = paths_HGG_train[int((len(paths_HGG_train)+1)*.1):int((len(paths_HGG_train)+1)*.2)]
# paths_HGG_train_2 = paths_HGG_train[int((len(paths_HGG_train)+1)*.2):int((len(paths_HGG_train)+1)*.3)]
# paths_HGG_train_3 = paths_HGG_train[int((len(paths_HGG_train)+1)*.3):int((len(paths_HGG_train)+1)*.4)]
# paths_HGG_train_4 = paths_HGG_train[int((len(paths_HGG_train)+1)*.4):int((len(paths_HGG_train)+1)*.5)]
# paths_HGG_train_5 = paths_HGG_train[int((len(paths_HGG_train)+1)*.5):int((len(paths_HGG_train)+1)*.6)]
# paths_HGG_train_6 = paths_HGG_train[int((len(paths_HGG_train)+1)*.6):int((len(paths_HGG_train)+1)*.7)]
# paths_HGG_train_7 = paths_HGG_train[int((len(paths_HGG_train)+1)*.7):int((len(paths_HGG_train)+1)*.8)]
# paths_HGG_train_8 = paths_HGG_train[int((len(paths_HGG_train)+1)*.8):int((len(paths_HGG_train)+1)*.9)]
# paths_HGG_train_9 = paths_HGG_train[int((len(paths_HGG_train)+1)*.9):]

paths_HGG_train_0 = paths_HGG_train[:int((len(paths_HGG_train)+1)*.2)]
paths_HGG_train_1 = paths_HGG_train[int((len(paths_HGG_train)+1)*.2):int((len(paths_HGG_train)+1)*.4)]
paths_HGG_train_2 = paths_HGG_train[int((len(paths_HGG_train)+1)*.4):int((len(paths_HGG_train)+1)*.6)]
paths_HGG_train_3 = paths_HGG_train[int((len(paths_HGG_train)+1)*.6):int((len(paths_HGG_train)+1)*.8)]
paths_HGG_train_4 = paths_HGG_train[int((len(paths_HGG_train)+1)*.8):]


with open(pathlst_HGG_train_0 , 'wb') as fp:
    pickle.dump(paths_HGG_train_0, fp)

with open(pathlst_HGG_train_1 , 'wb') as fp:
    pickle.dump(paths_HGG_train_1, fp)

with open(pathlst_HGG_train_2 , 'wb') as fp:
    pickle.dump(paths_HGG_train_2, fp)

with open(pathlst_HGG_train_3 , 'wb') as fp:
    pickle.dump(paths_HGG_train_3, fp)

with open(pathlst_HGG_train_4 , 'wb') as fp:
    pickle.dump(paths_HGG_train_4, fp)

# with open(pathlst_HGG_train_5 , 'wb') as fp:
#     pickle.dump(paths_HGG_train_5, fp)

# with open(pathlst_HGG_train_6 , 'wb') as fp:
#     pickle.dump(paths_HGG_train_6, fp)

# with open(pathlst_HGG_train_7 , 'wb') as fp:
#     pickle.dump(paths_HGG_train_7, fp)

# with open(pathlst_HGG_train_8 , 'wb') as fp:
#     pickle.dump(paths_HGG_train_8, fp)

# with open(pathlst_HGG_train_9 , 'wb') as fp:
#     pickle.dump(paths_HGG_train_9, fp)

In [ ]:
# Split HGG data into 1/4 to make it more manageable to run 
# paths_HGG_val_0 = paths_HGG_val[:int((len(paths_HGG_val)+1)*.1)]
# paths_HGG_val_1 = paths_HGG_val[int((len(paths_HGG_val)+1)*.1):int((len(paths_HGG_val)+1)*.2)]
# paths_HGG_val_2 = paths_HGG_val[int((len(paths_HGG_val)+1)*.2):int((len(paths_HGG_val)+1)*.3)]
# paths_HGG_val_3 = paths_HGG_val[int((len(paths_HGG_val)+1)*.3):int((len(paths_HGG_val)+1)*.4)]
# paths_HGG_val_4 = paths_HGG_val[int((len(paths_HGG_val)+1)*.4):int((len(paths_HGG_val)+1)*.5)]
# paths_HGG_val_5 = paths_HGG_val[int((len(paths_HGG_val)+1)*.5):int((len(paths_HGG_val)+1)*.6)]
# paths_HGG_val_6 = paths_HGG_val[int((len(paths_HGG_val)+1)*.6):int((len(paths_HGG_val)+1)*.7)]
# paths_HGG_val_7 = paths_HGG_val[int((len(paths_HGG_val)+1)*.7):int((len(paths_HGG_val)+1)*.8)]
# paths_HGG_val_8 = paths_HGG_val[int((len(paths_HGG_val)+1)*.8):int((len(paths_HGG_val)+1)*.9)]
# paths_HGG_val_9 = paths_HGG_val[int((len(paths_HGG_val)+1)*.9):]

paths_HGG_val_0 = paths_HGG_val[:int((len(paths_HGG_val)+1)*.2)]
paths_HGG_val_1 = paths_HGG_val[int((len(paths_HGG_val)+1)*.2):int((len(paths_HGG_val)+1)*.4)]
paths_HGG_val_2 = paths_HGG_val[int((len(paths_HGG_val)+1)*.4):int((len(paths_HGG_val)+1)*.6)]
paths_HGG_val_3 = paths_HGG_val[int((len(paths_HGG_val)+1)*.6):int((len(paths_HGG_val)+1)*.8)]
paths_HGG_val_4 = paths_HGG_val[int((len(paths_HGG_val)+1)*.8):]

with open(pathlst_HGG_val_0 , 'wb') as fp:
    pickle.dump(paths_HGG_val_0, fp)

with open(pathlst_HGG_val_1 , 'wb') as fp:
    pickle.dump(paths_HGG_val_1, fp)

with open(pathlst_HGG_val_2 , 'wb') as fp:
    pickle.dump(paths_HGG_val_2, fp)

with open(pathlst_HGG_val_3 , 'wb') as fp:
    pickle.dump(paths_HGG_val_3, fp)

with open(pathlst_HGG_val_4 , 'wb') as fp:
    pickle.dump(paths_HGG_val_4, fp)

# with open(pathlst_HGG_val_5 , 'wb') as fp:
#     pickle.dump(paths_HGG_val_5, fp)

# with open(pathlst_HGG_val_6 , 'wb') as fp:
#     pickle.dump(paths_HGG_val_6, fp)

# with open(pathlst_HGG_val_7 , 'wb') as fp:
#     pickle.dump(paths_HGG_val_7, fp)

# with open(pathlst_HGG_val_8 , 'wb') as fp:
#     pickle.dump(paths_HGG_val_8, fp)

# with open(pathlst_HGG_val_9 , 'wb') as fp:
#     pickle.dump(paths_HGG_val_9, fp)

In [ ]:
# Split HGG data into 1/4 to make it more manageable to run 
# paths_HGG_test_0 = paths_HGG_test[:int((len(paths_HGG_test)+1)*.1)]
# paths_HGG_test_1 = paths_HGG_test[int((len(paths_HGG_test)+1)*.1):int((len(paths_HGG_test)+1)*.2)]
# paths_HGG_test_2 = paths_HGG_test[int((len(paths_HGG_test)+1)*.2):int((len(paths_HGG_test)+1)*.3)]
# paths_HGG_test_3 = paths_HGG_test[int((len(paths_HGG_test)+1)*.3):int((len(paths_HGG_test)+1)*.4)]
# paths_HGG_test_4 = paths_HGG_test[int((len(paths_HGG_test)+1)*.4):int((len(paths_HGG_test)+1)*.5)]
# paths_HGG_test_5 = paths_HGG_test[int((len(paths_HGG_test)+1)*.5):int((len(paths_HGG_test)+1)*.6)]
# paths_HGG_test_6 = paths_HGG_test[int((len(paths_HGG_test)+1)*.6):int((len(paths_HGG_test)+1)*.7)]
# paths_HGG_test_7 = paths_HGG_test[int((len(paths_HGG_test)+1)*.7):int((len(paths_HGG_test)+1)*.8)]
# paths_HGG_test_8 = paths_HGG_test[int((len(paths_HGG_test)+1)*.8):int((len(paths_HGG_test)+1)*.9)]
# paths_HGG_test_9 = paths_HGG_test[int((len(paths_HGG_test)+1)*.9):]

paths_HGG_test_0 = paths_HGG_test[:int((len(paths_HGG_test)+1)*.2)]
paths_HGG_test_1 = paths_HGG_test[int((len(paths_HGG_test)+1)*.2):int((len(paths_HGG_test)+1)*.4)]
paths_HGG_test_2 = paths_HGG_test[int((len(paths_HGG_test)+1)*.4):int((len(paths_HGG_test)+1)*.6)]
paths_HGG_test_3 = paths_HGG_test[int((len(paths_HGG_test)+1)*.6):int((len(paths_HGG_test)+1)*.8)]
paths_HGG_test_4 = paths_HGG_test[int((len(paths_HGG_test)+1)*.8):]

with open(pathlst_HGG_test_0 , 'wb') as fp:
    pickle.dump(paths_HGG_test_0, fp)

with open(pathlst_HGG_test_1 , 'wb') as fp:
    pickle.dump(paths_HGG_test_1, fp)

with open(pathlst_HGG_test_2 , 'wb') as fp:
    pickle.dump(paths_HGG_test_2, fp)

with open(pathlst_HGG_test_3 , 'wb') as fp:
    pickle.dump(paths_HGG_test_3, fp)

with open(pathlst_HGG_test_4 , 'wb') as fp:
    pickle.dump(paths_HGG_test_4, fp)

# with open(pathlst_HGG_test_5 , 'wb') as fp:
#     pickle.dump(paths_HGG_test_5, fp)

# with open(pathlst_HGG_test_6 , 'wb') as fp:
#     pickle.dump(paths_HGG_test_6, fp)

# with open(pathlst_HGG_test_7 , 'wb') as fp:
#     pickle.dump(paths_HGG_test_7, fp)

# with open(pathlst_HGG_test_8 , 'wb') as fp:
#     pickle.dump(paths_HGG_test_8, fp)

# with open(pathlst_HGG_test_9 , 'wb') as fp:
#     pickle.dump(paths_HGG_test_9, fp)

# LGG Split Paths

In [ ]:
pathlst_LGG_train_0 = os.path.join(paths_path, "LGG_train_lst_0.pkl")
pathlst_LGG_train_1 = os.path.join(paths_path, "LGG_train_lst_1.pkl")
pathlst_LGG_train_2 = os.path.join(paths_path, "LGG_train_lst_2.pkl")
pathlst_LGG_train_3 = os.path.join(paths_path, "LGG_train_lst_3.pkl")
pathlst_LGG_train_4 = os.path.join(paths_path, "LGG_train_lst_4.pkl")
pathlst_LGG_train_5 = os.path.join(paths_path, "LGG_train_lst_5.pkl")
pathlst_LGG_train_6 = os.path.join(paths_path, "LGG_train_lst_6.pkl")
pathlst_LGG_train_7 = os.path.join(paths_path, "LGG_train_lst_7.pkl")
pathlst_LGG_train_8 = os.path.join(paths_path, "LGG_train_lst_8.pkl")
pathlst_LGG_train_9 = os.path.join(paths_path, "LGG_train_lst_9.pkl")

In [ ]:
pathlst_LGG_val_0 = os.path.join(paths_path, "LGG_val_lst_0.pkl")
pathlst_LGG_val_1 = os.path.join(paths_path, "LGG_val_lst_1.pkl")
pathlst_LGG_val_2 = os.path.join(paths_path,  "LGG_val_lst_2.pkl")
pathlst_LGG_val_3 = os.path.join(paths_path, "LGG_val_lst_3.pkl")
pathlst_LGG_val_4 = os.path.join(paths_path, "LGG_val_lst_4.pkl")
pathlst_LGG_val_5 = os.path.join(paths_path,  "LGG_val_lst_5.pkl")
pathlst_LGG_val_6 = os.path.join(paths_path, "LGG_val_lst_6.pkl")
pathlst_LGG_val_7 = os.path.join(paths_path, "LGG_val_lst_7.pkl")
pathlst_LGG_val_8 = os.path.join(paths_path,  "LGG_val_lst_8.pkl")
pathlst_LGG_val_9 = os.path.join(paths_path, "LGG_val_lst_9.pkl")

In [ ]:
pathlst_LGG_test_0 = os.path.join(paths_path, "LGG_test_lst_0.pkl")
pathlst_LGG_test_1 = os.path.join(paths_path, "LGG_test_lst_1.pkl")
pathlst_LGG_test_2 = os.path.join(paths_path,  "LGG_test_lst_2.pkl")
pathlst_LGG_test_3 = os.path.join(paths_path, "LGG_test_lst_3.pkl")
pathlst_LGG_test_4 = os.path.join(paths_path, "LGG_test_lst_4.pkl")
pathlst_LGG_test_5 = os.path.join(paths_path,  "LGG_test_lst_5.pkl")
pathlst_LGG_test_6 = os.path.join(paths_path, "LGG_test_lst_6.pkl")
pathlst_LGG_test_7 = os.path.join(paths_path, "LGG_test_lst_7.pkl")
pathlst_LGG_test_8 = os.path.join(paths_path,  "LGG_test_lst_8.pkl")
pathlst_LGG_test_9 = os.path.join(paths_path, "LGG_test_lst_9.pkl")

# Put LGG into files

In [ ]:
print("number of LGG train", len(paths_LGG_train))
print("number of LGG val", len(paths_LGG_val))
print("number of LGG test", len(paths_LGG_test))

number of LGG train 45
number of LGG val 15
number of LGG test 15


In [ ]:
# Split train into 60/20 for train/validation/test
random.shuffle(LGG_paths)

paths_LGG_train = LGG_paths[:int((len(LGG_paths)+1)*.60)]
paths_LGG_val = LGG_paths[int((len(LGG_paths)+1)*.60):int((len(LGG_paths)+1)*.80)]
paths_LGG_test = LGG_paths[int((len(LGG_paths)+1)*.80):]

In [ ]:
# Split LGG data into 1/4 to make it more manageable to run 
# paths_LGG_train_0 = paths_LGG_train[:int((len(paths_LGG_train)+1)*.1)]
# paths_LGG_train_1 = paths_LGG_train[int((len(paths_LGG_train)+1)*.1):int((len(paths_LGG_train)+1)*.2)]
# paths_LGG_train_2 = paths_LGG_train[int((len(paths_LGG_train)+1)*.2):int((len(paths_LGG_train)+1)*.3)]
# paths_LGG_train_3 = paths_LGG_train[int((len(paths_LGG_train)+1)*.3):int((len(paths_LGG_train)+1)*.4)]
# paths_LGG_train_4 = paths_LGG_train[int((len(paths_LGG_train)+1)*.4):int((len(paths_LGG_train)+1)*.5)]
# paths_LGG_train_5 = paths_LGG_train[int((len(paths_LGG_train)+1)*.5):int((len(paths_LGG_train)+1)*.6)]
# paths_LGG_train_6 = paths_LGG_train[int((len(paths_LGG_train)+1)*.6):int((len(paths_LGG_train)+1)*.7)]
# paths_LGG_train_7 = paths_LGG_train[int((len(paths_LGG_train)+1)*.7):int((len(paths_LGG_train)+1)*.8)]
# paths_LGG_train_8 = paths_LGG_train[int((len(paths_LGG_train)+1)*.8):int((len(paths_LGG_train)+1)*.9)]
# paths_LGG_train_9 = paths_LGG_train[int((len(paths_LGG_train)+1)*.9):]

paths_LGG_train_0 = paths_LGG_train[:int((len(paths_LGG_train)+1)*.1)]
paths_LGG_train_1 = paths_LGG_train[int((len(paths_LGG_train)+1)*.1):int((len(paths_LGG_train)+1)*.2)]
paths_LGG_train_2 = paths_LGG_train[int((len(paths_LGG_train)+1)*.2):int((len(paths_LGG_train)+1)*.3)]
paths_LGG_train_3 = paths_LGG_train[int((len(paths_LGG_train)+1)*.6):int((len(paths_LGG_train)+1)*.8)]
paths_LGG_train_4 = paths_LGG_train[int((len(paths_LGG_train)+1)*.8):]


with open(pathlst_LGG_train_0 , 'wb') as fp:
    pickle.dump(paths_LGG_train_0, fp)

with open(pathlst_LGG_train_1 , 'wb') as fp:
    pickle.dump(paths_LGG_train_1, fp)

with open(pathlst_LGG_train_2 , 'wb') as fp:
    pickle.dump(paths_LGG_train_2, fp)

with open(pathlst_LGG_train_3 , 'wb') as fp:
    pickle.dump(paths_LGG_train_3, fp)

with open(pathlst_LGG_train_4 , 'wb') as fp:
    pickle.dump(paths_LGG_train_4, fp)

with open(pathlst_LGG_train_5 , 'wb') as fp:
    pickle.dump(paths_LGG_train_5, fp)

with open(pathlst_LGG_train_6 , 'wb') as fp:
    pickle.dump(paths_LGG_train_6, fp)

with open(pathlst_LGG_train_7 , 'wb') as fp:
    pickle.dump(paths_LGG_train_7, fp)

with open(pathlst_LGG_train_8 , 'wb') as fp:
    pickle.dump(paths_LGG_train_8, fp)

with open(pathlst_LGG_train_9 , 'wb') as fp:
    pickle.dump(paths_LGG_train_9, fp)

In [ ]:
# Split HGG data into 1/4 to make it more manageable to run 
paths_LGG_val_0 = paths_LGG_val[:int((len(paths_LGG_val)+1)*.1)]
paths_LGG_val_1 = paths_LGG_val[int((len(paths_LGG_val)+1)*.1):int((len(paths_LGG_val)+1)*.2)]
paths_LGG_val_2 = paths_LGG_val[int((len(paths_LGG_val)+1)*.2):int((len(paths_LGG_val)+1)*.3)]
paths_LGG_val_3 = paths_LGG_val[int((len(paths_LGG_val)+1)*.3):int((len(paths_LGG_val)+1)*.4)]
paths_LGG_val_4 = paths_LGG_val[int((len(paths_LGG_val)+1)*.4):int((len(paths_LGG_val)+1)*.5)]
paths_LGG_val_5 = paths_LGG_val[int((len(paths_LGG_val)+1)*.5):int((len(paths_LGG_val)+1)*.6)]
paths_LGG_val_6 = paths_LGG_val[int((len(paths_LGG_val)+1)*.6):int((len(paths_LGG_val)+1)*.7)]
paths_LGG_val_7 = paths_LGG_val[int((len(paths_LGG_val)+1)*.7):int((len(paths_LGG_val)+1)*.8)]
paths_LGG_val_8 = paths_LGG_val[int((len(paths_LGG_val)+1)*.8):int((len(paths_LGG_val)+1)*.9)]
paths_LGG_val_9 = paths_LGG_val[int((len(paths_LGG_val)+1)*.9):]

with open(pathlst_LGG_val_0 , 'wb') as fp:
    pickle.dump(paths_LGG_val_0, fp)

with open(pathlst_LGG_val_1 , 'wb') as fp:
    pickle.dump(paths_LGG_val_1, fp)

with open(pathlst_LGG_val_2 , 'wb') as fp:
    pickle.dump(paths_LGG_val_2, fp)

with open(pathlst_LGG_val_3 , 'wb') as fp:
    pickle.dump(paths_LGG_val_3, fp)

with open(pathlst_LGG_val_4 , 'wb') as fp:
    pickle.dump(paths_LGG_val_4, fp)

with open(pathlst_LGG_val_5 , 'wb') as fp:
    pickle.dump(paths_LGG_val_5, fp)

with open(pathlst_LGG_val_6 , 'wb') as fp:
    pickle.dump(paths_LGG_val_6, fp)

with open(pathlst_LGG_val_7 , 'wb') as fp:
    pickle.dump(paths_LGG_val_7, fp)

with open(pathlst_LGG_val_8 , 'wb') as fp:
    pickle.dump(paths_LGG_val_8, fp)

with open(pathlst_LGG_val_9 , 'wb') as fp:
    pickle.dump(paths_LGG_val_9, fp)

In [ ]:
# Split HGG data into 1/4 to make it more manageable to run 
paths_LGG_test_0 = paths_LGG_test[:int((len(paths_LGG_test)+1)*.1)]
paths_LGG_test_1 = paths_LGG_test[int((len(paths_LGG_test)+1)*.1):int((len(paths_LGG_test)+1)*.2)]
paths_LGG_test_2 = paths_LGG_test[int((len(paths_LGG_test)+1)*.2):int((len(paths_LGG_test)+1)*.3)]
paths_LGG_test_3 = paths_LGG_test[int((len(paths_LGG_test)+1)*.3):int((len(paths_LGG_test)+1)*.4)]
paths_LGG_test_4 = paths_LGG_test[int((len(paths_LGG_test)+1)*.4):int((len(paths_LGG_test)+1)*.5)]
paths_LGG_test_5 = paths_LGG_test[int((len(paths_LGG_test)+1)*.5):int((len(paths_LGG_test)+1)*.6)]
paths_LGG_test_6 = paths_LGG_test[int((len(paths_LGG_test)+1)*.6):int((len(paths_LGG_test)+1)*.7)]
paths_LGG_test_7 = paths_LGG_test[int((len(paths_LGG_test)+1)*.7):int((len(paths_LGG_test)+1)*.8)]
paths_LGG_test_8 = paths_LGG_test[int((len(paths_LGG_test)+1)*.8):int((len(paths_LGG_test)+1)*.9)]
paths_LGG_test_9 = paths_LGG_test[int((len(paths_LGG_test)+1)*.9):]

with open(pathlst_LGG_test_0 , 'wb') as fp:
    pickle.dump(paths_LGG_test_0, fp)

with open(pathlst_LGG_test_1 , 'wb') as fp:
    pickle.dump(paths_LGG_test_1, fp)

with open(pathlst_LGG_test_2 , 'wb') as fp:
    pickle.dump(paths_LGG_test_2, fp)

with open(pathlst_LGG_test_3 , 'wb') as fp:
    pickle.dump(paths_LGG_test_3, fp)

with open(pathlst_LGG_test_4 , 'wb') as fp:
    pickle.dump(paths_LGG_test_4, fp)

with open(pathlst_LGG_test_5 , 'wb') as fp:
    pickle.dump(paths_LGG_test_5, fp)

with open(pathlst_LGG_test_6 , 'wb') as fp:
    pickle.dump(paths_LGG_test_6, fp)

with open(pathlst_LGG_test_7 , 'wb') as fp:
    pickle.dump(paths_LGG_test_7, fp)

with open(pathlst_LGG_val_8 , 'wb') as fp:
    pickle.dump(paths_LGG_val_8, fp)

with open(pathlst_LGG_val_9 , 'wb') as fp:
    pickle.dump(paths_LGG_val_9, fp)

# Processing HGG Train Data

In [ ]:
# Processing HGG Training Data
with open (pathlst_HGG_train_0, 'rb') as fp:
    paths_HGG_train_0 = pickle.load(fp)
    
HGG_train_data_0 = []
for path in paths_HGG_train_0:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_train_data_0.append(data)

In [ ]:
HGG_train_data = np.array(HGG_train_data_0)
HGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy","HGG_train_dataset_0.npy")
np.save(HGG_train_dataset_path, HGG_train_data)

In [ ]:
# Processing HGG Training Data
with open (pathlst_HGG_train_1, 'rb') as fp:
    paths_HGG_train_1 = pickle.load(fp)

HGG_train_data_1 = []
for path in paths_HGG_train_1:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_train_data_1.append(data)

In [ ]:
HGG_train_data = np.array(HGG_train_data_1)
HGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_train_dataset_1.npy")
np.save(HGG_train_dataset_path, HGG_train_data)

In [ ]:
# Processing HGG Training Data
with open (pathlst_HGG_train_2, 'rb') as fp:
    paths_HGG_train_2 = pickle.load(fp)

HGG_train_data_2 = []
for path in paths_HGG_train_2:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_train_data_2.append(data)

In [ ]:
HGG_train_data = np.array(HGG_train_data_2)
HGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_train_dataset_2.npy")
np.save(HGG_train_dataset_path, HGG_train_data)

In [ ]:
# Processing HGG Training Data
with open (pathlst_HGG_train_3, 'rb') as fp:
    paths_HGG_train_3 = pickle.load(fp)

HGG_train_data_3 = []
for path in paths_HGG_train_3:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_train_data_3.append(data)

In [ ]:
HGG_train_data = np.array(HGG_train_data_3)
HGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_train_dataset_3.npy")
np.save(HGG_train_dataset_path, HGG_train_data)

In [ ]:
# Processing HGG Training Data
with open (pathlst_HGG_train_4, 'rb') as fp:
    paths_HGG_train_4 = pickle.load(fp)

HGG_train_data_4 = []
for path in paths_HGG_train_4:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_train_data_4.append(data)

In [ ]:
HGG_train_data = np.array(HGG_train_data_4)
HGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_train_dataset_4.npy")
np.save(HGG_train_dataset_path, HGG_train_data)

In [ ]:
# Processing HGG Training Data
with open (pathlst_HGG_train_5, 'rb') as fp:
    paths_HGG_train_5 = pickle.load(fp)

HGG_train_data_5 = []
for path in paths_HGG_train_5:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_train_data_5.append(data)

In [ ]:
HGG_train_data = np.array(HGG_train_data_5)
HGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_train_dataset_5.npy")
np.save(HGG_train_dataset_path, HGG_train_data)

In [ ]:
# Processing HGG Training Data
with open (pathlst_HGG_train_6, 'rb') as fp:
    paths_HGG_train_6 = pickle.load(fp)

HGG_train_data_6 = []
for path in paths_HGG_train_6:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_train_data_6.append(data)

In [ ]:
HGG_train_data = np.array(HGG_train_data_6)
HGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_train_dataset_6.npy")
np.save(HGG_train_dataset_path, HGG_train_data)

In [ ]:
# Processing HGG Training Data
with open (pathlst_HGG_train_7, 'rb') as fp:
    paths_HGG_train_7 = pickle.load(fp)

HGG_train_data_7 = []
for path in paths_HGG_train_7:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_train_data_7.append(data)

In [ ]:
HGG_train_data = np.array(HGG_train_data_7)
HGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_train_dataset_7.npy")
np.save(HGG_train_dataset_path, HGG_train_data)

In [ ]:
# Processing HGG Training Data
with open (pathlst_HGG_train_8, 'rb') as fp:
    paths_HGG_train_8 = pickle.load(fp)

HGG_train_data_8 = []
for path in paths_HGG_train_8:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_train_data_8.append(data)

In [ ]:
HGG_train_data = np.array(HGG_train_data_8)
HGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_train_dataset_8.npy")
np.save(HGG_train_dataset_path, HGG_train_data)

In [ ]:
# Processing HGG Training Data
with open (pathlst_HGG_train_9, 'rb') as fp:
    paths_HGG_train_9 = pickle.load(fp)

HGG_train_data_9 = []
for path in paths_HGG_train_9:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_train_data_9.append(data)

In [ ]:
HGG_train_data = np.array(HGG_train_data_9)
HGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_train_dataset_9.npy")
np.save(HGG_train_dataset_path, HGG_train_data)

# Processing HGG Validation Data

In [ ]:
# Processing HGG Validation Data
with open (pathlst_HGG_val_0, 'rb') as fp:
    paths_HGG_val_0 = pickle.load(fp)
    
HGG_val_data_0 = []
for path in paths_HGG_val_0:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_val_data_0.append(data)

In [ ]:
HGG_val_data = np.array(HGG_val_data_0)
HGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_val_dataset_0.npy")
np.save(HGG_val_dataset_path, HGG_val_data)

In [ ]:
# Processing HGG Validation Data
with open (pathlst_HGG_val_1, 'rb') as fp:
    paths_HGG_val_1 = pickle.load(fp)
    
HGG_val_data_1 = []
for path in paths_HGG_val_1:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_val_data_1.append(data)

In [ ]:
HGG_val_data = np.array(HGG_val_data_1)
HGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_val_dataset_1.npy")
np.save(HGG_val_dataset_path, HGG_val_data)

In [ ]:
# Processing HGG Validation Data
with open (pathlst_HGG_val_2, 'rb') as fp:
    paths_HGG_val_2 = pickle.load(fp)
    
HGG_val_data_2 = []
for path in paths_HGG_val_2:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_val_data_2.append(data)

In [ ]:
HGG_val_data = np.array(HGG_val_data_2)
HGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_val_dataset_2.npy")
np.save(HGG_val_dataset_path, HGG_val_data)

In [ ]:
# Processing HGG Validation Data
with open (pathlst_HGG_val_3, 'rb') as fp:
    paths_HGG_val_3 = pickle.load(fp)
    
HGG_val_data_3 = []
for path in paths_HGG_val_3:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_val_data_3.append(data)

In [ ]:
HGG_val_data = np.array(HGG_val_data_3)
HGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_val_dataset_3.npy")
np.save(HGG_val_dataset_path, HGG_val_data)

In [ ]:
# Processing HGG Validation Data
with open (pathlst_HGG_val_4, 'rb') as fp:
    paths_HGG_val_4 = pickle.load(fp)
    
HGG_val_data_4= []
for path in paths_HGG_val_4:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_val_data_4.append(data)

In [ ]:
HGG_val_data = np.array(HGG_val_data_4)
HGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_val_dataset_4.npy")
np.save(HGG_val_dataset_path, HGG_val_data)

In [ ]:
# Processing HGG Validation Data
with open (pathlst_HGG_val_5, 'rb') as fp:
    paths_HGG_val_5 = pickle.load(fp)
    
HGG_val_data_5 = []
for path in paths_HGG_val_5:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_val_data_5.append(data)

In [ ]:
HGG_val_data = np.array(HGG_val_data_5)
HGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_val_dataset_5.npy")
np.save(HGG_val_dataset_path, HGG_val_data)

In [ ]:
# Processing HGG Validation Data
with open (pathlst_HGG_val_6, 'rb') as fp:
    paths_HGG_val_6 = pickle.load(fp)
    
HGG_val_data_6 = []
for path in paths_HGG_val_6:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_val_data_6.append(data)

In [ ]:
HGG_val_data = np.array(HGG_val_data_6)
HGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_val_dataset_6.npy")
np.save(HGG_val_dataset_path, HGG_val_data)

In [ ]:
# Processing HGG Validation Data
with open (pathlst_HGG_val_7, 'rb') as fp:
    paths_HGG_val_7 = pickle.load(fp)
    
HGG_val_data_7 = []
for path in paths_HGG_val_7:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_val_data_7.append(data)

In [ ]:
HGG_val_data = np.array(HGG_val_data_7)
HGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_val_dataset_7.npy")
np.save(HGG_val_dataset_path, HGG_val_data)

In [ ]:
# Processing HGG Validation Data
with open (pathlst_HGG_val_8, 'rb') as fp:
    paths_HGG_val_8 = pickle.load(fp)
    
HGG_val_data_8 = []
for path in paths_HGG_val_8:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_val_data_8.append(data)

In [ ]:
HGG_val_data = np.array(HGG_val_data_8)
HGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_val_dataset_8.npy")
np.save(HGG_val_dataset_path, HGG_val_data)

In [ ]:
# Processing HGG Validation Data
with open (pathlst_HGG_val_9, 'rb') as fp:
    paths_HGG_val_9 = pickle.load(fp)
    
HGG_val_data_9 = []
for path in paths_HGG_val_9:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  HGG_val_data_9.append(data)

In [ ]:
HGG_val_data = np.array(HGG_val_data_9)
HGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "HGG_val_dataset_9.npy")
np.save(HGG_val_dataset_path, HGG_val_data)

# Processing LGG Train Data

In [ ]:
# Processing LGG Training Data
with open (pathlst_LGG_train_0, 'rb') as fp:
    paths_LGG_train_0 = pickle.load(fp)
    
LGG_train_data_0 = []
for path in paths_LGG_train_0:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_train_data_0.append(data)

In [ ]:
LGG_train_data = np.array(LGG_train_data_0)
LGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_train_dataset_0.npy")
np.save(LGG_train_dataset_path, LGG_train_data)

In [ ]:
# Processing LGG Training Data
with open (pathlst_LGG_train_1, 'rb') as fp:
    paths_LGG_train_1 = pickle.load(fp)
    
LGG_train_data_1 = []
for path in paths_LGG_train_1:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_train_data_1.append(data)

In [ ]:
LGG_train_data = np.array(LGG_train_data_1)
LGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_train_dataset_1.npy")
np.save(LGG_train_dataset_path, LGG_train_data)

In [ ]:
# Processing LGG Training Data
with open (pathlst_LGG_train_2, 'rb') as fp:
    paths_LGG_train_2 = pickle.load(fp)
    
LGG_train_data_2 = []
for path in paths_LGG_train_2:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_train_data_2.append(data)

In [ ]:
LGG_train_data = np.array(LGG_train_data_2)
LGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_train_dataset_2.npy")
np.save(LGG_train_dataset_path, LGG_train_data)

In [ ]:
# Processing LGG Training Data
with open (pathlst_LGG_train_3, 'rb') as fp:
    paths_LGG_train_3 = pickle.load(fp)
    
LGG_train_data_3 = []
for path in paths_LGG_train_3:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_train_data_3.append(data)

In [ ]:
LGG_train_data = np.array(LGG_train_data_3)
LGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_train_dataset_3.npy")
np.save(LGG_train_dataset_path, LGG_train_data)

In [ ]:
# Processing LGG Training Data
with open (pathlst_LGG_train_4, 'rb') as fp:
    paths_LGG_train_4 = pickle.load(fp)
    
LGG_train_data_4 = []
for path in paths_LGG_train_4:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_train_data_4.append(data)

In [ ]:
LGG_train_data = np.array(LGG_train_data_4)
LGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_train_dataset_4.npy")
np.save(LGG_train_dataset_path, LGG_train_data)

In [ ]:
# Processing LGG Training Data
with open (pathlst_LGG_train_5, 'rb') as fp:
    paths_LGG_train_5 = pickle.load(fp)
    
LGG_train_data_5 = []
for path in paths_LGG_train_5:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_train_data_5.append(data)

In [ ]:
LGG_train_data = np.array(LGG_train_data_5)
LGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_train_dataset_5.npy")
np.save(LGG_train_dataset_path, LGG_train_data)

In [ ]:
# Processing LGG Training Data
with open (pathlst_LGG_train_6, 'rb') as fp:
    paths_LGG_train_6 = pickle.load(fp)
    
LGG_train_data_6 = []
for path in paths_LGG_train_6:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_train_data_6.append(data)

In [ ]:
LGG_train_data = np.array(LGG_train_data_6)
LGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_train_dataset_6.npy")
np.save(LGG_train_dataset_path, LGG_train_data)

In [ ]:
# Processing LGG Training Data
with open (pathlst_LGG_train_7, 'rb') as fp:
    paths_LGG_train_7 = pickle.load(fp)
    
LGG_train_data_7 = []
for path in paths_LGG_train_7:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_train_data_7.append(data)

In [ ]:
LGG_train_data = np.array(LGG_train_data_7)
LGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_train_dataset_7.npy")
np.save(LGG_train_dataset_path, LGG_train_data)

In [ ]:
# Processing LGG Training Data
with open (pathlst_LGG_train_8, 'rb') as fp:
    paths_LGG_train_8 = pickle.load(fp)
    
LGG_train_data_8 = []
for path in paths_LGG_train_8:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_train_data_8.append(data)

In [ ]:
LGG_train_data = np.array(LGG_train_data_8)
LGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_train_dataset_8.npy")
np.save(LGG_train_dataset_path, LGG_train_data)

In [ ]:
# Processing LGG Training Data
with open (pathlst_LGG_train_9, 'rb') as fp:
    paths_LGG_train_9 = pickle.load(fp)
    
LGG_train_data_9 = []
for path in paths_LGG_train_9:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_train_data_9.append(data)

In [ ]:
LGG_train_data = np.array(LGG_train_data_9)
LGG_train_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_train_dataset_9.npy")
np.save(LGG_train_dataset_path, LGG_train_data)

# Processing LGG Validation Data

In [ ]:
# Processing LGG Validation Data
with open (pathlst_LGG_val_0, 'rb') as fp:
    paths_LGG_val_0 = pickle.load(fp)
    
LGG_val_data_0 = []
for path in paths_LGG_val_0:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_val_data_0.append(data)

In [ ]:
LGG_val_data = np.array(LGG_val_data_0)
LGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_val_dataset_0.npy")
np.save(LGG_val_dataset_path, LGG_val_data)

In [ ]:
# Processing LGG Validation Data
with open (pathlst_LGG_val_1, 'rb') as fp:
    paths_LGG_val_1 = pickle.load(fp)
    
LGG_val_data_1 = []
for path in paths_LGG_val_1:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_val_data_1.append(data)

In [ ]:
LGG_val_data = np.array(LGG_val_data_1)
LGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy","LGG_val_dataset_1.npy")
np.save(LGG_val_dataset_path, LGG_val_data)

In [ ]:
# Processing LGG Validation Data
with open (pathlst_LGG_val_2, 'rb') as fp:
    paths_LGG_val_2 = pickle.load(fp)
    
LGG_val_data_2 = []
for path in paths_LGG_val_2:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_val_data_2.append(data)

In [ ]:
LGG_val_data = np.array(LGG_val_data_2)
LGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_val_dataset_2.npy")
np.save(LGG_val_dataset_path, LGG_val_data)

In [ ]:
# Processing LGG Validation Data
with open (pathlst_LGG_val_3, 'rb') as fp:
    paths_LGG_val_3 = pickle.load(fp)
    
LGG_val_data_3 = []
for path in paths_LGG_val_3:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_val_data_3.append(data)

In [ ]:
LGG_val_data = np.array(LGG_val_data_3)
LGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_val_dataset_3.npy")
np.save(LGG_val_dataset_path, LGG_val_data)

In [ ]:
# Processing LGG Validation Data
with open (pathlst_LGG_val_4, 'rb') as fp:
    paths_LGG_val_4 = pickle.load(fp)
    
LGG_val_data_4 = []
for path in paths_LGG_val_4:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_val_data_4.append(data)

In [ ]:
LGG_val_data = np.array(LGG_val_data_4)
LGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_val_dataset_4.npy")
np.save(LGG_val_dataset_path, LGG_val_data)

In [ ]:
# Processing LGG Validation Data
with open (pathlst_LGG_val_5, 'rb') as fp:
    paths_LGG_val_5 = pickle.load(fp)
    
LGG_val_data_5 = []
for path in paths_LGG_val_5:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_val_data_5.append(data)

In [ ]:
LGG_val_data = np.array(LGG_val_data_5)
LGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_val_dataset_5.npy")
np.save(LGG_val_dataset_path, LGG_val_data)

In [ ]:
# Processing LGG Validation Data
with open (pathlst_LGG_val_6, 'rb') as fp:
    paths_LGG_val_6 = pickle.load(fp)
    
LGG_val_data_6 = []
for path in paths_LGG_val_6:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_val_data_6.append(data)

In [ ]:
LGG_val_data = np.array(LGG_val_data_6)
LGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_val_dataset_6.npy")
np.save(LGG_val_dataset_path, LGG_val_data)

In [ ]:
# Processing LGG Validation Data
with open (pathlst_LGG_val_7, 'rb') as fp:
    paths_LGG_val_7 = pickle.load(fp)
    
LGG_val_data_7 = []
for path in paths_LGG_val_7:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_val_data_7.append(data)

In [ ]:
LGG_val_data = np.array(LGG_val_data_7)
LGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_val_dataset_7.npy")
np.save(LGG_val_dataset_path, LGG_val_data)

In [ ]:
# Processing LGG Validation Data
with open (pathlst_LGG_val_8, 'rb') as fp:
    paths_LGG_val_8 = pickle.load(fp)
    
LGG_val_data_8 = []
for path in paths_LGG_val_8:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_val_data_8.append(data)

In [ ]:
LGG_val_data = np.array(LGG_val_data_8)
LGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_val_dataset_8.npy")
np.save(LGG_val_dataset_path, LGG_val_data)

In [ ]:
# Processing LGG Validation Data
with open (pathlst_LGG_val_9, 'rb') as fp:
    paths_LGG_val_9 = pickle.load(fp)
    
LGG_val_data_9 = []
for path in paths_LGG_val_9:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  LGG_val_data_9.append(data)

In [ ]:
LGG_val_data = np.array(LGG_val_data_9)
LGG_val_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "LGG_val_dataset_9.npy")
np.save(LGG_val_dataset_path, LGG_val_data)

# Processing HGG Test Data

In [ ]:
# Processing Test Data
with open (pathlst_HGG_test_0, 'rb') as fp:
    paths_HGG_test_0 = pickle.load(fp)
    
test_data_HGG_0 = []
for path in paths_HGG_test_0:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_HGG_0.append(data)

In [ ]:
test_data = np.array(test_data_HGG_0)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy","test_dataset_HGG_0.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_HGG_test_1, 'rb') as fp:
    paths_HGG_test_1 = pickle.load(fp)
    
test_data_HGG_1 = []
for path in paths_HGG_test_1:
  print(path)
  if path == '/content/drive/MyDrive/Bioimaging/Bioimagining Project/data/BRATS2017/Brats17TrainingData/HGG/Brats17_2013_2_1/Brats17_2013_2_1_flair.nii':
    path = '/content/drive/MyDrive/Bioimaging/Bioimagining Project/data/BRATS2017/Brats17TrainingData/HGG/Brats17_2013_2_1'
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_HGG_1.append(data)

/content/drive/MyDrive/Bioimaging/Bioimagining Project/data/BRATS2017/Brats17TrainingData/HGG/Brats17_CBICA_AQD_1
/content/drive/MyDrive/Bioimaging/Bioimagining Project/data/BRATS2017/Brats17TrainingData/HGG/Brats17_TCIA_406_1
/content/drive/MyDrive/Bioimaging/Bioimagining Project/data/BRATS2017/Brats17TrainingData/HGG/Brats17_2013_2_1/Brats17_2013_2_1_flair.nii
/content/drive/MyDrive/Bioimaging/Bioimagining Project/data/BRATS2017/Brats17TrainingData/HGG/Brats17_CBICA_AUN_1


In [ ]:
test_data = np.array(test_data_HGG_1)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy","test_dataset_HGG_1.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_HGG_test_2, 'rb') as fp:
    paths_HGG_test_2 = pickle.load(fp)
    
test_data_HGG_2 = []
for path in paths_HGG_test_2:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_HGG_2.append(data)

In [ ]:
test_data = np.array(test_data_HGG_2)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy","test_dataset_HGG_2.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_HGG_test_3, 'rb') as fp:
    paths_HGG_test_3 = pickle.load(fp)
    
test_data_HGG_3 = []
for path in paths_HGG_test_3:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_HGG_3.append(data)

In [ ]:
test_data = np.array(test_data_HGG_3)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy","test_dataset_HGG_3.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_HGG_test_4, 'rb') as fp:
    paths_HGG_test_4 = pickle.load(fp)
    
test_data_HGG_4 = []
for path in paths_HGG_test_4:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_HGG_4.append(data)

In [ ]:
test_data = np.array(test_data_HGG_4)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy","test_dataset_HGG_4.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_HGG_test_5, 'rb') as fp:
    paths_HGG_test_5 = pickle.load(fp)
    
test_data_HGG_5 = []
for path in paths_HGG_test_5:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_HGG_5.append(data)

In [ ]:
test_data = np.array(test_data_HGG_5)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy","test_dataset_HGG_5.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_HGG_test_6, 'rb') as fp:
    paths_HGG_test_6 = pickle.load(fp)
    
test_data_HGG_6 = []
for path in paths_HGG_test_6:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_HGG_6.append(data)

In [ ]:
test_data = np.array(test_data_HGG_6)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy","test_dataset_HGG_6.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_HGG_test_7, 'rb') as fp:
    paths_HGG_test_7 = pickle.load(fp)
    
test_data_HGG_7 = []
for path in paths_HGG_test_7:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_HGG_7.append(data)

In [ ]:
test_data = np.array(test_data_HGG_7)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy","test_dataset_HGG_7.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_HGG_test_8, 'rb') as fp:
    paths_HGG_test_8 = pickle.load(fp)
    
test_data_HGG_8 = []
for path in paths_HGG_test_8:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_HGG_8.append(data)

In [ ]:
test_data = np.array(test_data_HGG_8)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy","test_dataset_HGG_8.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_HGG_test_9, 'rb') as fp:
    paths_HGG_test_9 = pickle.load(fp)
    
test_data_HGG_9 = []
for path in paths_HGG_test_9:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_HGG_9.append(data)

In [ ]:
test_data = np.array(test_data_HGG_9)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy","test_dataset_HGG_9.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Test opening npy file
example_test_open_file = np.load(test_dataset_path)
print(example_test_open_file.shape)

ValueError: ignored

# Processing LGG Test

In [ ]:
# Processing Test Data
with open (pathlst_LGG_test_0, 'rb') as fp:
    paths_LGG_test_0 = pickle.load(fp)
    
test_data_LGG_0 = []
for path in paths_LGG_test_0:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_LGG_0.append(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
test_data = np.array(test_data_LGG_0)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "test_dataset_LGG_0.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_LGG_test_1, 'rb') as fp:
    paths_LGG_test_1 = pickle.load(fp)
    
test_data_LGG_1 = []
for path in paths_LGG_test_1:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_LGG_1.append(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
test_data = np.array(test_data_LGG_1)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "test_dataset_LGG_1.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_LGG_test_2, 'rb') as fp:
    paths_LGG_test_2 = pickle.load(fp)
    
test_data_LGG_2 = []
for path in paths_LGG_test_2:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_LGG_2.append(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
test_data = np.array(test_data_LGG_2)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "test_dataset_LGG_2.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_LGG_test_3, 'rb') as fp:
    paths_LGG_test_3 = pickle.load(fp)
    
test_data_LGG_3 = []
for path in paths_LGG_test_3:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_LGG_3.append(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
test_data = np.array(test_data_LGG_3)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "test_dataset_LGG_3.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_LGG_test_4, 'rb') as fp:
    paths_LGG_test_4 = pickle.load(fp)
    
test_data_LGG_4 = []
for path in paths_LGG_test_4:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_LGG_4.append(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
test_data = np.array(test_data_LGG_4)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "test_dataset_LGG_4.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_LGG_test_5, 'rb') as fp:
    paths_LGG_test_5 = pickle.load(fp)
    
test_data_LGG_5 = []
for path in paths_LGG_test_5:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_LGG_5.append(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
test_data = np.array(test_data_LGG_5)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "test_dataset_LGG_5.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_LGG_test_6, 'rb') as fp:
    paths_LGG_test_6 = pickle.load(fp)
    
test_data_LGG_6 = []
for path in paths_LGG_test_6:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_LGG_6.append(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
test_data = np.array(test_data_LGG_6)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "test_dataset_LGG_6.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_LGG_test_7, 'rb') as fp:
    paths_LGG_test_7 = pickle.load(fp)
    
test_data_LGG_7 = []
for path in paths_LGG_test_7:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_LGG_7.append(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
test_data = np.array(test_data_LGG_7)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "test_dataset_LGG_7.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_LGG_test_8, 'rb') as fp:
    paths_LGG_test_8 = pickle.load(fp)
    
test_data_LGG_8 = []
for path in paths_LGG_test_8:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_LGG_8.append(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
test_data = np.array(test_data_LGG_8)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "test_dataset_LGG_8.npy")
np.save(test_dataset_path, test_data)

In [ ]:
# Processing Test Data
with open (pathlst_LGG_test_9, 'rb') as fp:
    paths_LGG_test_9 = pickle.load(fp)
    
test_data_LGG_9 = []
for path in paths_LGG_test_9:
  path = get_nii_path(path)
  data = process_nii_data(path)
  # data = remove_blanks(data)
  data = data.to_numpy()
  test_data_LGG_9.append(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
test_data = np.array(test_data_LGG_9)
test_dataset_path = os.path.join(base_path, "BRATS2017", "npy", "test_dataset_LGG_9.npy")
np.save(test_dataset_path, test_data)

# Processing Mix Test Data

In [29]:
# Processing Test Data
for i in range(num_split):
  with open (pathlst_mix_test[i], 'rb') as fp:
      paths_mix_test = pickle.load(fp)
      
  mix_test_data = np.empty((5, 240, 240, 0))
  for path in paths_mix_test:
    path = get_nii_path(path)
    data = process_nii_data(path)
    data = remove_blanks(data)
    data = data.to_numpy()
    mix_test_data = np.concatenate((mix_test_data, data), axis = 3)
  mix_test_data_X, mix_test_data_y = process_samples(mix_test_data)
  mix_test_data_X, mix_test_data_y = three_channels(mix_test_data_X, mix_test_data_y)
  mix_test_data_X = normalize_data(mix_test_data_X)
  test_X_path = os.path.join(base_path, "Unet_data_smaller", "mix_test", "test_X_mix_{}.npy".format(i))
  test_y_path = os.path.join(base_path, "Unet_data_smaller", "mix_test", "test_y_mix_{}.npy".format(i))
  np.save(test_X_path, mix_test_data_X)
  np.save(test_y_path, mix_test_data_y)
  mix_test_data, mix_test_data_X, mix_test_data_y = None, None, None

# Processing Mix Train Data

In [30]:
# Processing Train Data
for i in range(num_split):
  with open (pathlst_mix_train[i], 'rb') as fp:
      paths_mix_train = pickle.load(fp)
      
  mix_train_data = np.empty((5, 240, 240, 0))
  for path in paths_mix_train:
    path = get_nii_path(path)
    data = process_nii_data(path)
    data = remove_blanks(data)
    data = data.to_numpy()
    mix_train_data = np.concatenate((mix_train_data, data), axis = 3)
  mix_train_data_X, mix_train_data_y = process_samples(mix_train_data)
  mix_train_data_X, mix_train_data_y = three_channels(mix_train_data_X, mix_train_data_y)
  mix_train_data_X = normalize_data(mix_train_data_X)
  train_X_path = os.path.join(base_path, "Unet_data_smaller", "mix_train", "train_X_mix_{}.npy".format(i))
  train_y_path = os.path.join(base_path, "Unet_data_smaller", "mix_train", "train_y_mix_{}.npy".format(i))
  np.save(train_X_path, mix_train_data_X)
  np.save(train_y_path, mix_train_data_y)
  mix_train_data, mix_train_data_X, mix_train_data_y = None, None, None

# Processing Mix Validation Data

In [31]:
# Processing Validation Data
for i in range(num_split):
  with open (pathlst_mix_val[i], 'rb') as fp:
      paths_mix_val = pickle.load(fp)
      
  mix_val_data = np.empty((5, 240, 240, 0))
  for path in paths_mix_val:
    path = get_nii_path(path)
    data = process_nii_data(path)
    data = remove_blanks(data)
    data = data.to_numpy()
    mix_val_data = np.concatenate((mix_val_data, data), axis = 3)
  mix_val_data_X, mix_val_data_y = process_samples(mix_val_data)
  mix_val_data_X, mix_val_data_y = three_channels(mix_val_data_X, mix_val_data_y)
  mix_val_data_X = normalize_data(mix_val_data_X)
  val_X_path = os.path.join(base_path, "Unet_data_smaller", "mix_val", "val_X_mix_{}.npy".format(i))
  val_y_path = os.path.join(base_path, "Unet_data_smaller", "mix_val", "val_y_mix_{}.npy".format(i))
  np.save(val_X_path, mix_val_data_X)
  np.save(val_y_path, mix_val_data_y)
  mix_val_data, mix_val_data_X, mix_val_data_y = None, None, None